In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS geodata.bronze;

CREATE TABLE IF NOT EXISTS geodata.bronze.well_header_occ;

COPY INTO geodata.bronze.well_header_occ
FROM 's3://databricks-purrio/raw/kingfisher_well_header_occ.csv' 
FILEFORMAT = CSV 
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true') 
COPY_OPTIONS ('mergeSchema' = 'true');

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
0,0,0


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS geodata.bronze;

CREATE TABLE IF NOT EXISTS geodata.bronze.well_header_sp;

COPY INTO geodata.bronze.well_header_sp
FROM 's3://databricks-purrio/raw/kingfisher_well_header_sp.csv' 
FILEFORMAT = CSV 
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true') 
COPY_OPTIONS ('mergeSchema' = 'true');

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5803418513770504>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', "CREATE SCHEMA IF NOT EXISTS geodata.bronze;\n\nCREATE TABLE IF NOT EXISTS geodata.bronze.well_header_sp;\n\nCOPY INTO geodata.bronze.well_header_sp\nFROM 's3://databricks-purrio/raw/kingfisher_well_header_sp.csv' \nFILEFORMAT = CSV \nFORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true') \nCOPY_OPTIONS ('mergeSchema' = 'true');\n")

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the

In [0]:
%sql
drop table if exists geodata.bronze.well_header_sp;
--CREATE TABLE IF NOT EXISTS geodata.bronze.well_header_sp;


CREATE TABLE IF NOT EXISTS geodata.bronze.well_header_sp
USING DELTA
TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

COPY INTO geodata.bronze.well_header_sp
FROM 's3://databricks-purrio/raw/kingfisher_well_header_sp.csv' 
FILEFORMAT = CSV 
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true') 
COPY_OPTIONS ('mergeSchema' = 'true');

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
10228,10228,0


In [0]:
# Column headers contain spaces in the well_header report format from S&P. This causes a DELTA_INVALID_CHARACTERS_IN_COLUMN_NAMES using the SQL option as show above. This alternative is idempotent and allows us to rename headers to replace spaces.

# Read the CSV file from S3
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("s3://databricks-purrio/raw/kingfisher_well_header_sp.csv")

# Replace spaces with underscores in column names
for col in df.columns:
    df = df.withColumnRenamed(col, col.replace(" ", "_"))

# Write the DataFrame to a Delta table with schema merge enabled
df.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("geodata.bronze.well_header_sp")


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS geodata.bronze;

CREATE TABLE IF NOT EXISTS geodata.bronze.well_header_enverus;

COPY INTO geodata.bronze.well_header_enverus
FROM 's3://databricks-purrio/raw/kingfisher_well_header_enverus.csv' 
FILEFORMAT = CSV 
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true') 
COPY_OPTIONS ('mergeSchema' = 'true');

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
9090,9090,0


In [0]:
%sql
--select count(*) as occ_count from geodata.bronze.well_header_occ;
select count(*) as sp_count from geodata.bronze.well_header_sp;
--select count(*) as enverus_count from geodata.bronze.well_header_enverus;

sp_count
10228


In [0]:
%sql
show tables in geodata.bronze;